In [ ]:
!pip install 'networkx<2.7'
!pip install transformers
import os
import numpy as np
import pandas as pd
import nltk
import re
import networkx as nx
import math
from transformers import pipeline
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from numpy import nan
#nltk.download('punkt') # one time execution
#nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.0 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.8 MB/s eta 0:00:00


In [ ]:
nltk.download('punkt') # one time execution
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
#!mkdir IS_Project
%cd IS_Project


Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/IS_Project


In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip

In [ ]:
# Extract word vectors
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
stop_words = stopwords.words('english')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
df = pd.read_csv("Goa_Limited_Attraction_Reviews.csv")
city = "Goa"
summaries = []
att_no = 0
unique_attractions1 = df['Tourist Attraction'].unique()
unique_attractions = [item for item in unique_attractions1 if (item == item) == True]
for attraction in unique_attractions:
  att_no += 1
  #print(att_no, attraction)
  Truth_Array = (df['Tourist Attraction'] == attraction)
  sentences = []
  #print(sum(Truth_Array))
  counter=0
  for review in df['Review'][Truth_Array]:
    #print(review)
    counter += 1
    #print(counter)
    if counter > 30:
      break
    #if len(sent_tokenize(review)) > 5:
      #sentences.append(sent_tokenize(review)[:5])
    #else:
      #sentences.append(sent_tokenize(review))
    sentences.append(sent_tokenize(review))
  sentences = [y for x in sentences for y in x]
  #print(sentences)
  # remove punctuations, numbers and special characters
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ", regex = True)
  # make alphabets lowercase
  clean_sentences = [s.lower() for s in clean_sentences]
  # remove stopwords from the sentences
  clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
  sentence_vectors = []
  for i in clean_sentences:
    if len(i) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)
  # similarity matrix
  sim_mat = np.zeros([len(sentences), len(sentences)])
  from sklearn.metrics.pairwise import cosine_similarity
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    #print(i)
  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  # Extract top 10 sentences as the summary
  stringy = []
  for i in range(min(6,len(sentences))):
    #print(ranked_sentences[i][1])
    stringy.append(ranked_sentences[i][1])
  string = " ".join(stringy)
  summary_text = summarizer(string, max_length= 60, min_length=5, do_sample=False)[0]['summary_text']
  summaries.append([attraction,summary_text])
  print("")
  print(att_no, attraction)
  print(summary_text)
  print("")
df = pd.DataFrame(summaries)
df.rename(columns = {0:'Attraction', 1:'Summarised Review'}, inplace = True)
df.insert(0, "City", city , True)
df.to_csv(city + "_Summarised_Reviews.csv", index = False)


1 Palolem_Beach-Canacona_South
i highly recommend you to visit this beach if you are in south goa . the only place that we found with beach side huts and amazing food .


2 Baga_Beach-Baga_North
the best is to stay in south goa for half the time and switch to baga for the end of the vacation . the beach is very overcrowded but nice place to spend time and relax.


3 Benaulim_Beach-Benaulim_Salcette_South
the beach is beautiful, clean and very less crowded . we went for dolphin trip and we're lucky to sight 6 dolphins close by.


4 Cavelossim_Beach-Cavelossim_South
the cavelossim beach is a delight to be in with family, clean waters and very good lifeguards too...the point where the sea meets the Sal River is breathtaking and a must see .


5 Agonda_Beach-Agonda_South
beautiful beach with very less people around . one can enjoy the clear waters and sunbath all day long .


6 Basilica_of_Bom_Jesus
Located in old goa, this Basilica has an amazing architecture and is spread on a large com